Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 2.3 ms, sys: 3.29 ms, total: 5.59 ms
Wall time: 14.1 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.104/10168/1 Dashboard: http://10.20.0.104:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 346 ms, sys: 24.9 ms, total: 370 ms
Wall time: 2.81 s


[1.1851631852798068,
 1.4752334507104328,
 1.6043971003608404,
 1.1731750270761523,
 1.7191972117621588,
 1.6443723777519397,
 1.5270524257739515,
 1.2249262654305442,
 1.3412744547817996,
 1.1726767121075272,
 1.259251622393101,
 1.7649860624670715,
 1.6792089992983494,
 1.892416032362367,
 1.1543490260903315,
 1.646175840513152,
 1.5979881836399532,
 1.5247825469374434,
 1.0704272590196582,
 1.664980390834799,
 1.5021254954348273,
 1.3750055457942123,
 1.6256191526052786,
 1.902179541084512,
 1.530991746058448,
 1.908856633213536,
 1.9638194813624597,
 1.2337233953022944,
 1.4518554916595603,
 1.2896378514026443,
 1.7305103894203004,
 1.1897653170541256,
 1.332429406101072,
 1.3456422415447546,
 1.3723421086500824,
 1.1993662115425718,
 1.1118898387643368,
 1.8261172022859717,
 1.3504979966642465,
 1.1318382867798924,
 1.354044626243212,
 1.356930281194681,
 1.0513479882361287,
 1.640403578087801,
 1.1962622477262026,
 1.2594971435521143,
 1.3225131618596446,
 1.6567256525758438,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.7773757841117468,
 1.6718136909264494,
 1.6118643887764077,
 1.6435081770495623,
 1.7805054394767552,
 1.7166095813584494,
 1.5354256192933216,
 1.7773696800804466,
 1.7681959796377023,
 1.798222362018382,
 1.3053383237517504,
 1.4808840285271716,
 1.103276952143534,
 1.948802160728886,
 1.8464424726166677,
 1.2004405233362516,
 1.8767441246617063,
 1.8263818690872509,
 1.4034474855154084,
 1.2068284630329402,
 1.1585150753704967,
 1.0589160381931546,
 1.3307327854887823,
 1.7146178568414059,
 1.832549488959304,
 1.2131984391417663,
 1.9805819861669227,
 1.6794280685809393,
 1.4216556554057584,
 1.0887117535829707,
 1.8161037202332344,
 1.6375863146629315,
 1.718595525475495,
 1.675415068759961,
 1.838279325090522,
 1.9062645285540567,
 1.4818336315628924,
 1.9729223769326047,
 1.2249347467514773,
 1.043859053131361,
 1.4036120691391674,
 1.4368642475258504,
 1.4773295595118037,
 1.1496437350197106,
 1.6864000542871804,
 1.6940508170307422,
 1.6679420632690136,
 1.0086405960124538,


3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 252 ms, sys: 18.9 ms, total: 271 ms
Wall time: 2.28 s
